In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 20
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
aa = [1] * 5
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, score:-286.59
 Observe Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:66.1, cum_e:66.1, new_i:59.1, cum_i:59.1
 True State Time: 1 | S: 25495749.8, E: 184.8, A: 0.0, I: 59.1, D: 0.0, R: 6.4, Budget: 2700000.00, score:-692.94
 Observe Time: 1 | S: 25495746.0, E: 179.0, A: 0.0, I: 59.1, D: 0.0, R: 0.0
 Time: 1 | new_e:92.9, cum_e:159.0, new_i:81.7, cum_i:140.8
 True State Time: 2 | S: 25495595.0, E: 249.5, A: 0.0, I: 140.7, D: 0.1, R: 14.7, Budget: 2700000.00, score:-1270.49
 Observe Time: 2 | S: 25495596.0, E: 249.0, A: 0.0, I: 140.7, D: 0.1, R: 0.0
 Time: 2 | new_e:131.1, cum_e:290.1, new_i:114.2, cum_i:255.0
 True State Time: 3 | S: 25495376.5, E: 342.5, A: 0.0, I: 254.7, D: 0.2, R: 26.0, Budget: 2700000.00, score:-2092.76
 Observe Time: 3 | S: 25495384.0, E: 340.0, A: 0.0, I: 254.7, D: 0.2, R: 15.0
 Time: 3 | new_e:185.9, cum_e:476.0, new_i:160.6, cum

pyomo fail
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 22 | S: 24370698.7, E: 36.8, A: 292.6, I: 974.3, D: 5.9, R: 1123991.7, Budget: 1913473.78, score:-13273.32
 Observe Time: 22 | S: 24370936.0, E: 33.0, A: 292.6, I: 974.3, D: 5.9, R: 1123991.0
 Time: 22 | new_e:16.6, cum_e:2979.9, new_i:46.3, cum_i:3969.6
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 23 | S: 24296031.7, E: 16.6, A: 279.1, I: 898.8, D: 6.0, R: 1198767.9, Budget: 1854006.99, score:-13312.25
 Observe Time: 23 | S

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 41 | S: 23313502.3, E: 7.2, A: 42.0, I: 188.0, D: 7.2, R: 2182253.3, Budget: 1210836.19, score:-13891.54
 Observe Time: 41 | S: 23313828.0, E: 0.0, A: 42.0, I: 188.0, D: 7.2, R: 2182264.0
 Time: 41 | new_e:9.9, cum_e:3105.1, new_i:11.8, cum_i:4312.4
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 42 | S: 23259796.4, E: 13.1, A: 38.4, I: 181.0, D: 7.3, R: 2235963.9, Budget: 1173168.05, score:-13917.35
 Observe Time: 42 | S: 23260126.0, E: 2.0, A: 38.4, I: 181.0, D: 7.3, R: 2235974.0
 Time: 42 | new_e:5.6, cum_e:3110.7, new_i:9.3, cum_i:4321.7
 True State Time: 43 | S: 23209137.7, E: 9.6, A: 39.7, I: 167.6, D: 7.3, R: 2286638.

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 58 | S: 22560046.7, E: 10.2, A: 20.4, I: 71.0, D: 7.5, R: 2935844.1, Budget: 720179.14, score:-14237.91
 Observe Time: 58 | S: 22560462.0, E: 0.0, A: 20.4, I: 71.0, D: 7.5, R: 2935860.0
 Time: 58 | new_e:6.1, cum_e:3180.9, new_i:7.3, cum_i:4429.9
 True State Time: 59 | S: 22522352.7, E: 12.3, A: 19.8, I: 70.6, D: 7.6, R: 2973537.1, Budget: 693437.44, score:-14260.51
 Observe Time: 59 | S: 22522773.0, E: 1.0, A: 19.8, I: 70.6, D: 7.6, R: 2973556.0
 Time: 59 | new_e:5.1, cum_e:3186.0, new_i:6.8, cum_i:4436.7
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 60 | S: 22485916.8, E: 11.6, A: 20.6, I: 68.6, D: 7.6, R: 3009974.7, Bud

 True State Time: 76 | S: 21995274.2, E: 97.8, A: 29.0, I: 154.2, D: 7.8, R: 3500437.0, Budget: 373079.91, score:-15458.83
 Observe Time: 76 | S: 21996042.0, E: 0.0, A: 29.0, I: 154.2, D: 7.8, R: 3500451.0
 Time: 76 | new_e:44.7, cum_e:3458.8, new_i:43.4, cum_i:4726.0
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 77 | S: 21963062.1, E: 122.3, A: 29.9, I: 182.3, D: 7.8, R: 3532595.6, Budget: 355155.16, score:-15675.08
 Observe Time: 77 | S: 21963895.0, E: 3.0, A: 29.9, I: 182.3, D: 7.8, R: 3532609.0
 Time: 77 | new_e:49.6, cum_e:3508.4, new_i:49.4, cum_i:4775.3
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - messa

In [6]:
#result_state = pd.DataFrame(state_buffer)
#result_state.to_csv(r'MPC_state_16.csv')
#result_new = pd.DataFrame(new_buffer)
#result_new.to_csv(r'MPC_new_16.csv')

In [7]:
(time.time() - tic) / ( T - start_time + 1)

131.5462279880748

In [8]:
(70507-20500)/70507

0.7092487270767441